In [1]:
import pandas as pd
import glob

from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

## Read Data

In [2]:
path = 'C:/Users/sutantoy/Downloads/rooroo/csv'
all_files = glob.glob(path + "/*.csv")

li = []

for file in all_files:
    file_df = pd.read_csv(file)
    
    area = file.split('\\')[1].replace('.csv','')
    file_df['area'] = area
    
    li.append(file_df)

df = pd.concat(li, axis=0, ignore_index=True)
df.drop(columns=['block', 'street name', 'lease commence year', 'resale registration date'], inplace=True)
# lease commentment year corr to remaining lease

mapping = {'01 to 03': 2, 
           '04 to 06': 5, 
           '07 to 09': 8, 
           '10 to 12': 11, 
           '13 to 15': 14, 
           '16 to 18': 17, 
           '19 to 21': 20,
           '22 to 24': 23,
           '25 to 27': 26,
           '28 to 30': 29,
           '31 to 33': 36,
           '34 to 36': 35,
           '37 to 39': 38,
           '40 to 42': 41,
           '43 to 45': 44,
           '46 to 48': 47}
df['storey'] = df['storey'].map(mapping)

df['remaining lease (yr)'] = df['remaining lease (yr)'].astype(float)

df = df[df['storey'] >=16]

display(df.head())
print(df.shape)

,storey,floor area (sqm),flat model,remaining lease (yr),resale price ($),area
74,26,70,Model A,90.00,530000,Ang Mo Kio
225,26,70,Model A,90.33,543000,Ang Mo Kio
299,17,67,Model A,93.67,455000,Ang Mo Kio
342,20,67,Model A,93.75,473000,Ang Mo Kio
344,29,70,Model A,90.58,510000,Ang Mo Kio


(564, 6)


In [3]:
df.corr()

,storey,floor area (sqm),remaining lease (yr),resale price ($)
storey,1.000000,-0.158993,0.547079,0.607290
floor area (sqm),-0.158993,1.000000,-0.182966,-0.080822
remaining lease (yr),0.547079,-0.182966,1.000000,0.836310
resale price ($),0.607290,-0.080822,0.836310,1.000000


In [4]:
x = df.drop(columns = 'resale price ($)')
x = pd.get_dummies(x)
y = df[['resale price ($)']]

In [5]:
reg = sm.OLS(y,x)

In [6]:
reg.fit().summary2().tables

[                     0                 1                    2           3
 0               Model:               OLS      Adj. R-squared:       0.895
 1  Dependent Variable:  resale price ($)                 AIC:  13549.2138
 2                Date:  2021-11-26 15:14                 BIC:  13622.9097
 3    No. Observations:               564      Log-Likelihood:     -6757.6
 4            Df Model:                16         F-statistic:       302.3
 5        Df Residuals:               547  Prob (F-statistic):   2.84e-259
 6           R-squared:             0.898               Scale:  1.5413e+09,
                                       Coef.      Std.Err.          t  \
 storey                          1834.121502    290.643426   6.310556   
 floor area (sqm)                4054.860144    465.868013   8.703882   
 remaining lease (yr)            4136.527087    420.595517   9.834929   
 flat model_Dbss               -47037.169954  31356.102509  -1.500096   
 flat model_Improved          -128